# FAKE REVIEW NODE EMBEDDING

In [1]:
# importing all necessary libraries
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import os
import networkx as nx
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
#from node2vec import Node2Vec as n2v
sns.set()

%matplotlib inline

In [2]:
import os, sys
from copy import deepcopy
import numpy as np
import pandas as pd

import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
print(f"Torch version: {th.__version__}.  CUDA version: {th.version.cuda}")

Torch version: 1.12.1+cu113.  CUDA version: 11.3


In [4]:
dgl_version = f"dgl-cu{th.version.cuda.replace('.', '')}==0.7.2"
dgl_version

'dgl-cu113==0.7.2'

In [5]:
# Follow instructions at https://www.dgl.ai/pages/start.html
!{sys.executable} -m pip install {dgl_version} -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 152.9 MB 35 kB/s 


In [6]:
os.environ['DGLBACKEND'] = 'pytorch'
import dgl
from dgl import function as fn

Using backend: pytorch


In [7]:
dgl.__version__

'0.7.2'

In [8]:
from dgl.dataloading.pytorch import NodeDataLoader
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [9]:
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import argparse

from dgl.sampling import node2vec_random_walk

In [10]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
#nltk.download('all')
import re
pd.set_option('max_colwidth', 800)


#importing Libraies
import networkx as nx
import matplotlib.pyplot as plt
from numpy import genfromtxt
import collections
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression,LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [11]:
#loading dataset
df=pd.read_csv('/content/category 1.csv')
df.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfortable. I love it!Very pretty"
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I've had mine for a couple of years"
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and feel of this pillow.
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it is a great product for the price! I"
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the set for two months now and have not been


In [12]:
#df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [13]:
df.shape

(4056, 4)

In [14]:
df['label'].value_counts()

CG    2028
OR    2028
Name: label, dtype: int64

In [15]:
# dropping unnecessary columns. 
df.drop(['category', 'rating'], axis = 1, inplace=True)
df.head()

,label,text_
0,CG,"Love this! Well made, sturdy, and very comfortable. I love it!Very pretty"
1,CG,"love it, a great upgrade from the original. I've had mine for a couple of years"
2,CG,This pillow saved my back. I love the look and feel of this pillow.
3,CG,"Missing information on how to use it, but it is a great product for the price! I"
4,CG,Very nice set. Good quality. We have had the set for two months now and have not been


In [16]:
z = {'OR' : 1, 'CG' : 0}  # 1 for original reviews and 0 for computer generated reviews
df['label'] = df['label'].map(z)

In [17]:
# Cleaning Raw tweets
def clean_text(text):
    
    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])
    
    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces 
    text = re.sub('\s+', ' ', text)
    
    return text

In [18]:
# applying that into function.
df["clean_text"] = df['text_'].apply(lambda x: clean_text(x))
df.head(10)

,label,text_,clean_text
0,0,"Love this! Well made, sturdy, and very comfortable. I love it!Very pretty",Love this Well made sturdy and very comfortable I love it Very pretty
1,0,"love it, a great upgrade from the original. I've had mine for a couple of years",love it a great upgrade from the original I ve had mine for a couple of years
2,0,This pillow saved my back. I love the look and feel of this pillow.,This pillow saved my back I love the look and feel of this pillow
3,0,"Missing information on how to use it, but it is a great product for the price! I",Missing information on how to use it but it is a great product for the price I
4,0,Very nice set. Good quality. We have had the set for two months now and have not been,Very nice set Good quality We have had the set for two months now and have not been
5,0,I WANTED DIFFERENT FLAVORS BUT THEY ARE NOT.,I WANTED DIFFERENT FLAVORS BUT THEY ARE NOT
6,0,They are the perfect touch for me and the only thing I wish they had a little more space.,They are the perfect touch for me and the only thing I wish they had a little more space
7,0,These done fit well and look great. I love the smoothness of the edges and the extra,These done fit well and look great I love the smoothness of the edges and the extra
8,0,"Great big numbers & easy to read, the only thing I didn't like is the size of the",Great big numbers easy to read the only thing I didn t like is the size of the
9,0,My son loves this comforter and it is very well made. We also have a baby,My son loves this comforter and it is very well made We also have a baby


In [19]:
# dropping the uncleaned text column "text"
df = df.drop(['text_'], axis=1)

In [20]:
# renaming the column
df.rename(columns = {'clean_text':'text', 'label': 'Class'}, inplace = True)
df.head()

,Class,text
0,0,Love this Well made sturdy and very comfortable I love it Very pretty
1,0,love it a great upgrade from the original I ve had mine for a couple of years
2,0,This pillow saved my back I love the look and feel of this pillow
3,0,Missing information on how to use it but it is a great product for the price I
4,0,Very nice set Good quality We have had the set for two months now and have not been


In [21]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

{'’m', '‘d', 'why', 'off', 'this', 'one', 'enough', 'these', 'could', 'fifteen', 'yourself', 'ten', 'move', 'whom', 'seems', 'mostly', 'almost', 'when', 'its', 'rather', 'before', 'wherein', 'or', 'somewhere', 'therein', 'twenty', 'me', '’d', 'except', 'were', 'either', 'anyhow', 'most', 'whence', 'moreover', 'she', 'everywhere', 'her', 'throughout', 'alone', 'latterly', 'six', 'sometime', 'against', 'hers', 'top', 'after', 'as', 'above', 'besides', 'put', 'seemed', 'via', 'side', "n't", 'your', 'without', 'whatever', 'whose', 'ca', 'yours', '‘s', 'very', 'regarding', 'already', 'whither', 'is', 'wherever', 'fifty', 'others', 'more', 'behind', 'thru', 'was', 'now', 'where', 'few', 'along', 'once', 'thence', 'forty', 'he', 'whereby', '’s', '‘ve', 'it', 'three', 'and', 'get', 'would', 'both', 'own', 'hereupon', 'herself', 'made', 'us', 'else', 'if', 'around', 'seeming', 'used', '’ve', 'even', 'myself', 'n’t', 'an', 'otherwise', 'the', 'then', 'upon', 'something', 'than', 'least', 'only',

In [22]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [23]:
sw_nltk.extend(['th','stay'])
print(len(sw_nltk))

181


In [24]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
#stop_words = set(stopwords.words('english'))
ps=PorterStemmer()
wnl=WordNetLemmatizer()
def clean_lemma(review):
    review = review.lower()
    review = review.split()
    review = [wnl.lemmatize(word) for word in review if word not in sw_nltk]
    review = [wnl.lemmatize(word) for word in review if word not in sw_spacy]
    review = ' '.join(review)
    return review

In [25]:
# using lemmatization
df['text']=df['text'].apply(clean_lemma)
df.head()

,Class,text
0,0,love sturdy comfortable love pretty
1,0,love great upgrade original couple year
2,0,pillow saved love look feel pillow
3,0,missing information use great product price
4,0,nice set good quality set month


### FEATURE EXTRACTION

In [26]:
x = df['text']
y = df['Class']

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer()
corpii=cv.fit_transform(x) 

In [28]:
feature_names = cv.get_feature_names()
#feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [29]:
len(feature_names)

7007

In [30]:
u1 = pd.DataFrame(corpii.toarray(), columns=feature_names)
u1.head()

,aa,aaa,ab,abbey,ability,able,abruptly,absolute,absolutely,absorb,...,zip,ziploc,zipper,ziti,zojirushi,zoo,zoodles,zucchini,zuchinni,zyliss
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### FEATURE SELECTION

In [32]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Two features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k = 800)
X_kbest_features = chi2_features.fit_transform(u1, y)

In [33]:
chi_support = chi2_features.get_support()

In [34]:
chi_feature = u1.loc[:,chi_support].columns.tolist()

In [35]:
chi_feature

['absorbent',
 'accent',
 'access',
 'actually',
 'add',
 'added',
 'addition',
 'adjustable',
 'admit',
 'ago',
 'air',
 'allow',
 'amazon',
 'america',
 'american',
 'anticipated',
 'anymore',
 'apart',
 'appear',
 'appearance',
 'applied',
 'apply',
 'appreciate',
 'area',
 'arm',
 'arthritic',
 'asap',
 'ask',
 'assembled',
 'assembly',
 'attached',
 'attachment',
 'attention',
 'available',
 'avid',
 'avoid',
 'away',
 'bacon',
 'bad',
 'balloon',
 'band',
 'bar',
 'bare',
 'barely',
 'based',
 'basement',
 'basically',
 'bath',
 'bathroom',
 'battery',
 'bean',
 'beat',
 'beater',
 'beautifully',
 'bed',
 'beer',
 'begin',
 'believe',
 'believer',
 'belt',
 'bent',
 'best',
 'big',
 'bit',
 'black',
 'blade',
 'blanket',
 'blowing',
 'blue',
 'board',
 'body',
 'boiling',
 'bold',
 'bolt',
 'booklet',
 'bother',
 'bottle',
 'bought',
 'bracket',
 'brand',
 'breville',
 'brew',
 'bride',
 'bright',
 'broken',
 'brush',
 'bubble',
 'build',
 'bulb',
 'burn',
 'butter',
 'button',
 

In [36]:
X_kbest_features

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.29540252,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.13240486,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.08588805,
        0.        ]])

In [37]:
u3 = pd.DataFrame(X_kbest_features, columns = chi_feature)
u3.head()

,absorbent,accent,access,actually,add,added,addition,adjustable,admit,ago,...,wobble,woman,wonder,wooden,working,wrap,wrinkle,write,year,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.295403,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [38]:
# SOURCE : https://github.com/dmlc/dgl/issues/3364
src, dst = np.nonzero(X_kbest_features)

In [39]:
src

array([   0,    0,    0, ..., 4055, 4055, 4055])

In [40]:
dst

array([136, 416, 538, ..., 761, 789, 798])

In [41]:
g = dgl.graph((src, dst))

In [42]:
n_nodes = g.num_nodes()
n_nodes

4056

In [43]:
nodes = g.nodes()
nodes

tensor([   0,    1,    2,  ..., 4053, 4054, 4055])

In [44]:
n_nodes1 = g.number_of_nodes()
n_nodes1

4056

In [45]:
g.ndata['feat'] = torch.tensor(X_kbest_features)
g.ndata['feat'] 

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2954, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1324, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0859, 0.0000]],
       dtype=torch.float64)

In [46]:
n_train = int(n_nodes1 * 0.6)
n_val = int(n_nodes1 * 0.2)

In [47]:
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(train_mask)
train_mask

tensor([False, False, False,  ..., False, False, False])

In [48]:
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(val_mask)
val_mask

tensor([False, False, False,  ..., False, False, False])

In [49]:
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(test_mask)
test_mask

tensor([False, False, False,  ..., False, False, False])

In [50]:
train_mask[:n_train] = True

In [51]:
val_mask[n_train:n_train + n_val] = True

In [52]:
test_mask[n_train + n_val:] = True

In [53]:
g.ndata['train_mask'] = train_mask

In [54]:
g.ndata['val_mask'] = val_mask

In [55]:
g.ndata['test_mask'] = test_mask

In [56]:
s_array = df['Class'].to_numpy()
s_array

array([0, 0, 0, ..., 1, 0, 1])

In [57]:
t1 = torch.tensor(s_array)
t1

tensor([0, 0, 0,  ..., 1, 0, 1])

In [58]:
g.ndata["label"] =t1

In [59]:
label = g.ndata["label"]

In [60]:
label[train_mask]

tensor([0, 0, 0,  ..., 0, 0, 0])

In [61]:
label[val_mask]

tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [62]:
label[test_mask]

tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,

In [63]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [64]:
val_mask

tensor([False, False, False,  ..., False, False, False])

In [65]:
test_mask

tensor([False, False, False,  ...,  True,  True,  True])

In [66]:
clf = LogisticRegression()
clf.fit(g.ndata['feat'][train_mask], label[train_mask])

#y_hat_val_lr = clf.predict_proba(X_kbest_features[val_mask,:])
y_val_lr = clf.predict(g.ndata['feat'][val_mask])

In [67]:
accuracy_score(label[val_mask], y_val_lr)

0.8927250308261405

In [68]:
clf_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
clf_balanced.fit(g.ndata['feat'][train_mask], label[train_mask])

y_val_lrb = clf_balanced.predict(g.ndata['feat'][val_mask])
accuracy_score(label[val_mask], y_val_lrb)

0.8902589395807645

In [69]:
rf = RandomForestClassifier(n_estimators=400)
rf.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = rf.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.812577065351418

In [70]:
gb = GradientBoostingClassifier(n_estimators=300, learning_rate=1.0, max_depth=1, random_state=0)
gb.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = gb.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.782983970406905

In [71]:
ab = AdaBoostClassifier(n_estimators=400, random_state=0)
ab.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = ab.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.7435265104808878

In [72]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=7, shuffle=True)

In [73]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter=10000) 
scores = cross_val_score(log, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8508295323237853


In [74]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC()
scores = cross_val_score(lsvm, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8740083926290824


In [75]:
from sklearn.linear_model import PassiveAggressiveClassifier
pa=PassiveAggressiveClassifier(max_iter=1000, random_state=7, tol=1e-3)
scores = cross_val_score(pa, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8577291248555616


In [76]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rf, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8348135984917594


In [77]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=300)
scores = cross_val_score(ab, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8229781670011553


In [78]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=300)
scores = cross_val_score(gb, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8412138904092927


In [79]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, 
              gamma=0,  importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, 
             n_estimators=200, n_jobs=16,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, subsample=1,
              tree_method='exact', use_label_encoder=False
              )
scores = cross_val_score(xgbc, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8525664416469014


In [80]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [81]:
node_features = g.ndata['feat'].float()
node_labels = g.ndata['label']
train_mask = g.ndata['train_mask']
valid_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [82]:
node_features

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2954, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1324, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0859, 0.0000]])

In [83]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [84]:
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    model.train()
    # forward propagation by using all nodes
    logits = model(g, node_features)
    # compute loss
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # compute validation accuracy
    acc = evaluate(model, g, node_features, node_labels, test_mask)
    print(acc)
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    #print(loss.item())

    # Save model if necessary.  Omitted in this example.

0.47783251231527096


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


0.5455665024630542
0.6022167487684729
0.6600985221674877
0.7179802955665024
0.7733990147783252
0.8054187192118226
0.8312807881773399
0.8608374384236454
0.8731527093596059
0.8842364532019704
0.896551724137931
0.9014778325123153
0.9051724137931034
0.9113300492610837
0.9174876847290641
0.9224137931034483
0.9248768472906403
0.9248768472906403
0.9248768472906403
0.9285714285714286
0.9310344827586207
0.9322660098522167
0.9347290640394089
0.937192118226601
0.9384236453201971
0.9396551724137931
0.937192118226601
0.9384236453201971
0.9396551724137931
0.9396551724137931
0.9384236453201971
0.937192118226601
0.9347290640394089
0.9359605911330049
0.937192118226601
0.937192118226601
0.937192118226601
0.9396551724137931
0.9396551724137931
0.9384236453201971
0.9384236453201971
0.9384236453201971
0.9384236453201971
0.9396551724137931
0.9396551724137931
0.937192118226601
0.937192118226601
0.937192118226601
0.937192118226601
